# Imports

In [1]:
import sys
sys.path.append('../../../')

from utils.create_and_prepare_model_qlora import (
    create_and_prepare_LLaMA_model
)

import torch
# import bitsandbytes as bnb
import datetime
from peft import (
    PeftModel,
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)

from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    BitsAndBytesConfig
)
import transformers
from datasets import load_dataset, Dataset 

import pandas as pd


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


------

# Functions

## Load Model and Tokenizer

## Create Dataset

In [2]:
def get_dataset(path):
    instruction = 'Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante'
    messages = {}
    messages['instruction'] =[]
    messages['input'] = []
    messages['output'] = [] 
    
    df = pd.read_csv(path, skip_blank_lines=True)
    df = df.dropna(how='all')
    for _, linha in df.iterrows():
        messages['instruction'].append(instruction)
        messages['input'].append(linha['comentario'])
        messages['output'].append(linha['sugestaoResposta'])
            
    return Dataset.from_dict(messages)

## Create Prompt

In [3]:
def generate_prompt(data_point):
    prompt = f'''\
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point['instruction']}
### Input:
{data_point['input']}
### Response:
{data_point['output']}
'''.strip()               
    return prompt

## Tokenize Prompts

In [4]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
        return_tensors=None
    )
    
    ## adicionando eos token ao final dos tokens
    if (
        result['input_ids'][-1] != tokenizer.eos_token_id
        and len(result['input_ids']) < MAX_LENGTH
        and ADD_EOS_TOKEN
    ):
        result['input_ids'].append(tokenizer.eos_token_id)
        result['attention_mask'].append(1)
        
    # generative models: labels are the same as the input
    result['labels'] = result['input_ids'].copy()
    
    return result

## Generate and Tokenize the prompts

In [5]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_prompt = tokenize(full_prompt)
    return tokenized_prompt

------

# Defining Parameters

In [6]:
model_name = 'meta-llama/Llama-2-7b-hf'
train_path = '../../../data/base_2k/train_base.csv'
validation_path = '../../../data/base_2k/validation_base.csv'    
OUTPUT_DIR = "./results/qlora_LLaMA_2_7B_transformer_trainer"+datetime.datetime.now().isoformat()

MAX_LENGTH=350
ADD_EOS_TOKEN=True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_quant_type="nf4",
#     bnb_8bit_compute_dtype=torch.bfloat16,
#     bnb_8bit_use_double_quant=True,
# )

peft_config =  LoraConfig(
    lora_alpha=16, # 16
    lora_dropout=0.1, # 0.05
    bias="none",
    task_type="CAUSAL_LM",
    r=64, # 8
    target_modules=[
        "q_proj",
        "v_proj",
        # "k_proj",
        # "o_proj"
    ]
)

training_arguments = transformers.TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4, # micro_batch_size
    gradient_accumulation_steps=4, # batch_size / micro_batch_size, # per_device_train_batch_size * gradient_accumulation_steps = 16
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    # max_steps=100,
    # save_steps=100,
    optim="paged_adamw_32bit", # adamw_torch
    save_total_limit=2,
    logging_steps=10,
    learning_rate=2e-4, # 3e-4
    # fp16=True,
    bf16=True, # testar
    max_grad_norm=0.3,
    warmup_ratio=0.03, #0.05
    # warmup_steps=100,
    lr_scheduler_type="constant", #cosine
    # evaluation_strategy='steps',
    eval_steps=20,
    # load_best_model_at_end=True,
    save_strategy = "epoch",
    evaluation_strategy="steps",
    report_to="wandb"
)


----

# Run

## Loading Model and Tokenizer

In [7]:
model, tokenizer = create_and_prepare_LLaMA_model(model_name, peft_config, bnb_config)

tokenizer.pad_token = (
    0
)

tokenizer.padding_side = "left"
model.config.use_cache = False
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 33,554,432 || all params: 3,533,967,360 || trainable%: 0.9494833591219133


## Defining Data collator

In [8]:
# pad_to_multiple_of -> An integer representing the maximum sequence length, rounded up to the nearest multiple of this value.
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

## Preprocessing the datasets

In [9]:
train_data = get_dataset(train_path)
validation_data = get_dataset(validation_path)

train_data = train_data.map(generate_and_tokenize_prompt)
validation_data = validation_data.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/1729 [00:00<?, ? examples/s]

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

## Trainer

In [10]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    args=training_arguments,
    data_collator=data_collator,
)

## Training

In [11]:
# old_state_dict = model.state_dict
# model.state_dict = (
#     lambda self, *_, **__: get_peft_model_state_dict(
#         self, old_state_dict()
#     )
# ).__get__(model, type(model))
# model = torch.compile(model)

In [12]:
trainer.train()
model.save_pretrained(OUTPUT_DIR)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: matheusalb. Use `wandb login --relogin` to force relogin


  0%|          | 0/108 [00:00<?, ?it/s]

{'loss': 1.9319, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 1.3393, 'learning_rate': 0.0002, 'epoch': 0.18}


  0%|          | 0/54 [00:00<?, ?it/s]

{'eval_loss': 1.135132908821106, 'eval_runtime': 31.3829, 'eval_samples_per_second': 6.883, 'eval_steps_per_second': 1.721, 'epoch': 0.18}
{'loss': 1.1369, 'learning_rate': 0.0002, 'epoch': 0.28}
{'loss': 1.0129, 'learning_rate': 0.0002, 'epoch': 0.37}


...

{'eval_loss': 0.8547875881195068, 'eval_runtime': 31.9615, 'eval_samples_per_second': 6.758, 'eval_steps_per_second': 1.69, 'epoch': 0.55}
{'loss': 0.8561, 'learning_rate': 0.0002, 'epoch': 0.65}
{'loss': 0.8309, 'learning_rate': 0.0002, 'epoch': 0.74}


  0%|          | 0/54 [00:00<?, ?it/s]

{'eval_loss': 0.8276867866516113, 'eval_runtime': 31.7902, 'eval_samples_per_second': 6.795, 'eval_steps_per_second': 1.699, 'epoch': 0.74}
{'loss': 0.7898, 'learning_rate': 0.0002, 'epoch': 0.83}
{'loss': 0.8274, 'learning_rate': 0.0002, 'epoch': 0.92}


  0%|          | 0/54 [00:00<?, ?it/s]

{'eval_loss': 0.8160731196403503, 'eval_runtime': 27.2516, 'eval_samples_per_second': 7.926, 'eval_steps_per_second': 1.982, 'epoch': 0.92}
{'train_runtime': 969.58, 'train_samples_per_second': 1.783, 'train_steps_per_second': 0.111, 'train_loss': 1.0435464558777985, 'epoch': 1.0}


In [13]:
trainer.save_model(OUTPUT_DIR+'/trainer_save_model')
tokenizer.save_pretrained(OUTPUT_DIR+'/tokenizer')


('./results/qlora_LLaMA_2_7B_transformer_trainer2023-08-01T00:26:37.257542/tokenizer/tokenizer_config.json',
 './results/qlora_LLaMA_2_7B_transformer_trainer2023-08-01T00:26:37.257542/tokenizer/special_tokens_map.json',
 './results/qlora_LLaMA_2_7B_transformer_trainer2023-08-01T00:26:37.257542/tokenizer/tokenizer.model',
 './results/qlora_LLaMA_2_7B_transformer_trainer2023-08-01T00:26:37.257542/tokenizer/added_tokens.json')

# Inferencia

In [14]:
def gen_input(comment):
    prompt = f'''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
{comment}
### Response:
'''           
    return prompt

In [15]:
generation_config=model.generation_config
generation_config.max_new_tokens=300
# generation_config.temeperature=0.7
# generation_config.top_k=0.9
generation_config.num_return_sequences=1
# generation_config.pad_token_id=tokenizer.eos_token_id
# generation_config.eos_token_id=tokenizer.eos_token_id

def inference(model, text):
    enconded = tokenizer(text, return_tensors="pt")
    outputs = model.generate(
        input_ids=enconded.input_ids,
        attention_mask=enconded.attention_mask,
        generation_config=generation_config
    )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [16]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Alimentação caríssima para um péssimo atendimento.. super mal atendido
### Response:
Caro cliente, lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo atendimento insatisfatório e pela alta cobrança. Vamos revisar nossos processos internos para garantir que isso não aconteça novamente. Agradecemos seu feedback e esperamos ter a oportunidade de oferecer uma experiência melhor no futuro.


In [17]:
#treino
comment='Filé estava duro. Ficamos decepcionados.'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Filé estava duro. Ficamos decepcionados.
### Response:
Caro cliente, lamentamos muito pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo filé que estava duro. Vamos investigar o ocorrido e tomar as medidas necessárias para garantir que isso não aconteça novamente. Agradecemos seu feedback e esperamos ter a oportunidade de recebê-lo novamente em nosso estabelecimento.


In [18]:
comment='''A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.'''
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.
### Response:
Olá, agradecemos pelo seu feedback e lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pela falta de vagas para cadeirantes e idosos e pela falta de apoio dos nossos funcionários para ajudar na questão do cadeirante. Vamos revisar noss

In [19]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
### Response:
Caro cliente, lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo atendimento lento e pela confusão com o prato que você pediu. Vamos investigar o ocorrido para garantir que isso não se repita. Agradecemos seu feedback e esperamos ter a oportunidade de oferecer uma experiência melhor no futuro.


In [20]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
### Response:
Caro cliente, lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo atendimento lento e pela confusão com o prato que você pediu. Vamos investigar o ocorrido para garantir que isso não se repita. Agradecemos seu feedback e esperamos ter a oportunidade de oferecer uma experiência melhor no futuro.


In [21]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
### Response:
Olá, agradecemos pelo seu feedback. Lamentamos que a temperatura do nosso restaurante não tenha sido adequada para sua comodidade. Vamos avaliar nossos sistemas de climatização para garantir que isso não aconteça novamente. Quanto à demora na preparação da comida, entendemos que isso pode ter causado uma má experiência. Vamos trabalhar para melhorar nossos processos de produção para garantir que isso não aconteça novamente. Agradecemos seu feedback e esperamos ter a oportunidade de recebê-la novamente em nosso restaurante.


In [22]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
### Response:
Olá, agradecemos pelo seu feedback e entendemos sua frustração em relação ao tempo de espera para ser atendido na área externa. Lamentamos que isso tenha ocorrido e estamos trabalhando para melhorar nossa organização para atender todos os clientes de forma equilibrada. Agradecemos seu apoio e esperamos ter a oportunidade de recebê-la novamente em nosso restaurante.


In [23]:
comment = 'O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!
### Response:
Caro cliente, lamentamos profundamente pela experiência negativa que você teve com nosso yakisoba. Pedimos desculpas pelo péssimo sabor do molho e pela massa amassada. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos seu feedback e esperamos ter a oportunidade de oferecer uma experiência melhor no futuro.


In [24]:
comment= '''Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas ,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento .
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.'''
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas ,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento .
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.
### Response:
Caro cliente, lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo atendimento insatisfatório e pela demora no preparo do seu omelete. Vamos investigar o ocorrido e tomar as medidas necessárias para garantir que isso não se repita. Agradecemos seu feedback e esperamos ter a oportu

In [25]:
#treino
comment='Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!'

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!
### Response:
Caro cliente,

Lamentamos p

In [26]:
#treino
comment = '''Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.'''
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.
### Response:
Olá, agradecemos pelo seu feedback sobre a limpeza da nossa panificadora. Lamentamos que você tenha tido uma experiência negativa em relação à higiene. Vamos revisar nossos processos de limpeza para garantir que todos os pontos da nossa estabelecimento sejam limpos e seguros. Agradecemos seu apoio e esperamos r

In [27]:
comment = '''Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer seu carater?
Tudo bem que a mão de obra de estrangeiros seja mais barata. Mas vocês teriam que pensar no bem estar dos clientes.
Descobri o nome do garçom.
CARLOS AMED. Ele é cubano.
Ia fazer uma denuncia por assedio mas pensei na empresa. Por favor selecione melhor seus colaboradores.
Muito indignada.'''

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer

In [28]:
comment = "Fui no dia 10/12/21 às 11:00 o restaurante não tinha ninguém ainda, fui fazer um pedido para viagem. Depois de quase 30 minutos de espera percebo que tinham clientes que chegaram depois comendo...questionei um garçom pela demora do pedido foi averiguar e não deu nenhuma satisfação. Aí percebi depois que o pedido estava no balcão aguardando o favor de algum garçom pegar e olha que nem estava lotado o restaurante...desrespeito com o cliente e garçons mal treinados.. nunca mais nesse lugar."
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Fui no dia 10/12/21 às 11:00 o restaurante não tinha ninguém ainda, fui fazer um pedido para viagem. Depois de quase 30 minutos de espera percebo que tinham clientes que chegaram depois comendo...questionei um garçom pela demora do pedido foi averiguar e não deu nenhuma satisfação. Aí percebi depois que o pedido estava no balcão aguardando o favor de algum garçom pegar e olha que nem estava lotado o restaurante...desrespeito com o cliente e garçons mal treinados.. nunca mais nesse lugar.
### Response:
Caro cliente, lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pela demora no entrega do seu pedido e pela falta de atenção dos nossos garçons. Va

In [29]:
comment = "Só não dou zero porque não dá e porque a torta de frango estava boazinha. Priorizam quem tem aparência de ter mais dinheiro e os garçons não fazem questão alguma de atender as pessoas. Olharam na minha cara, passaram direto até que eu implorasse pra ser atendida. :) Não anotaram meu pedido e ainda queriam colocar a culpa em mim pelo esquecimento. Enfim, não volto!"
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Só não dou zero porque não dá e porque a torta de frango estava boazinha. Priorizam quem tem aparência de ter mais dinheiro e os garçons não fazem questão alguma de atender as pessoas. Olharam na minha cara, passaram direto até que eu implorasse pra ser atendida. :) Não anotaram meu pedido e ainda queriam colocar a culpa em mim pelo esquecimento. Enfim, não volto!
### Response:
Olá, agradecemos pelo seu feedback e lamentamos que sua experiência não tenha sido satisfatória. Pedimos desculpas pela falta de atenção e pela falta de respeito aos clientes. Vamos revisar nossos processos internos para garantir que isso não aconteça novamente. Quanto ao pedido, entendemos sua frustração e vamos investigar o 

In [30]:
comment = "Pedimos um prato de bife ao molho madeira sendo que o feijão veio com gosto de molho madeira e o bife veio com gosto de queijo então eu recomendaria que vocês se dedicassem mais e as atende vistas são muito de humor baixo"

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Pedimos um prato de bife ao molho madeira sendo que o feijão veio com gosto de molho madeira e o bife veio com gosto de queijo então eu recomendaria que vocês se dedicassem mais e as atende vistas são muito de humor baixo
### Response:
Olá, agradecemos pelo seu feedback sobre a sua experiência em nosso restaurante. Lamentamos muito que o prato de bife ao molho madeira não tenha atendido às suas expectativas. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Quanto às atende vistas, vamos trabalhar para melhorar a qualidade do nosso serviço. Agradecemos por nos informar e esperamos ter a oportunidade de recebê-la novamente em nosso estabelecimento.


In [31]:
comment = "Pedi no ifood camarão empanado e mandaram camarão alho e óleo muito sem graça e poucas unidades. Fui inventar de experimentar por causa da propaganda no Instagram e me ferrei! Não recomendo."

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Pedi no ifood camarão empanado e mandaram camarão alho e óleo muito sem graça e poucas unidades. Fui inventar de experimentar por causa da propaganda no Instagram e me ferrei! Não recomendo.
### Response:
Olá, agradecemos pelo seu feedback. Lamentamos muito pela experiência negativa que você teve com nosso camarão empanado. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por nos informar e esperamos ter a oportunidade de recebê-la novamente em nosso restaurante.
